In [1]:
import pandas as pd
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D , BatchNormalization , Dropout , Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import TensorBoard , ModelCheckpoint , LearningRateScheduler

In [2]:
data_generator = ImageDataGenerator(validation_split=0.25,
                                    rescale=1./255)

In [3]:
#BATCH_SIZE = 64 # 집 PC
BATCH_SIZE = 8 #회사 Main PC

In [4]:
train_gen = data_generator.flow_from_directory("./Dataset/Adience_Dataset/Crop_WB_Adjust/Train/",
                                               target_size = (224, 224),
                                               batch_size = BATCH_SIZE,  
                                               subset="training",
                                               class_mode = 'categorical'
                                              )

Found 10161 images belonging to 2 classes.


In [5]:
valid_gen = data_generator.flow_from_directory("./Dataset/Adience_Dataset/Crop_WB_Adjust/Train/",
                                               target_size = (224, 224),
                                               batch_size = BATCH_SIZE,  
                                               subset="validation",
                                               class_mode = 'categorical'
                                              )

Found 3385 images belonging to 2 classes.


In [6]:
log_dir = os.path.join('Logs')
CHECKPOINT_PATH = os.path.join('CheckPoints')
tb_callback = TensorBoard(log_dir=log_dir)

cp = ModelCheckpoint(filepath=CHECKPOINT_PATH, monitor='val_categorical_accuracy',
                     save_best_only = True,
                     verbose = 1)

In [7]:
initial_learning_rate = 0.01

def lr_exp_decay(epoch, lr):
    k = 0.1
    return initial_learning_rate * np.math.exp(-k*epoch)

In [8]:
lr_scheduler = LearningRateScheduler(lr_exp_decay, verbose=1)

In [9]:
EfficientNet = tf.keras.applications.efficientnet.EfficientNetB0(
    #weights='imagenet',
    weights=None,
    input_shape=(224, 224, 3),
    include_top=False)

In [10]:
#EfficientNet.trainable = False

In [11]:
DROP_OUT_RATE = 0.25

In [12]:
model= Sequential()
model.add( EfficientNet )
model.add( GlobalAveragePooling2D() ) 
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(512, activation='relu') )
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(128, activation='relu') )
model.add( Dropout( DROP_OUT_RATE ) ) 
model.add( BatchNormalization() ) 
model.add( Dense(2, activation='softmax') )

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 7, 7, 1280)        4049571   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1280)              5120      
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2

In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy'],
)

In [15]:
hist = model.fit(train_gen,
                 epochs = 50,
                 validation_data=valid_gen,
                 callbacks=[cp , tb_callback , lr_scheduler],
                 verbose = 1 
)

C:\Users\csyi\Anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 1/50

Epoch 00001: LearningRateScheduler reducing learning rate to 0.01.
1271/1271 [==============================] - 394s 301ms/step - loss: 0.7484 - categorical_accuracy: 0.5087 - val_loss: 0.6878 - val_categorical_accuracy: 0.5563

Epoch 00001: val_categorical_accuracy improved from -inf to 0.55628, saving model to CheckPoints


C:\Users\csyi\Anaconda3\envs\TF.2.5.0-GPU\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


INFO:tensorflow:Assets written to: CheckPoints\assets
Epoch 2/50

Epoch 00002: LearningRateScheduler reducing learning rate to 0.009048374180359595.
1271/1271 [==============================] - 372s 293ms/step - loss: 0.6952 - categorical_accuracy: 0.5204 - val_loss: 0.6936 - val_categorical_accuracy: 0.5291

Epoch 00002: val_categorical_accuracy did not improve from 0.55628
Epoch 3/50

Epoch 00003: LearningRateScheduler reducing learning rate to 0.008187307530779819.
1271/1271 [==============================] - 374s 294ms/step - loss: 0.6930 - categorical_accuracy: 0.5253 - val_loss: 0.6932 - val_categorical_accuracy: 0.5013

Epoch 00003: val_categorical_accuracy did not improve from 0.55628
Epoch 4/50

Epoch 00004: LearningRateScheduler reducing learning rate to 0.007408182206817179.
1271/1271 [==============================] - 373s 294ms/step - loss: 0.6930 - categorical_accuracy: 0.5225 - val_loss: 0.6939 - val_categorical_accuracy: 0.4848

Epoch 00004: val_categorical_accuracy did

KeyboardInterrupt: 